In [1]:
#Importing Packages
using Polymake #Via polymake.jl
using Test
using Base.Threads  # For parallel threads

polymake version 4.9
Copyright (c) 1997-2023
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
#Checking Polymake works properly
p = polytope.Polytope(POINTS=[1 -1 -1; 1 1 -1; 1 -1 1; 1 1 1; 1 0 0])
@show p.VERTICES
@show p.N_LATTICE_POINTS

p.VERTICES = pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1

p.N_LATTICE_POINTS = 9


9

In [3]:
MAX_LATTICE_POINTS = 44  # global variable for the maximum number of lattice points
polytopes_list = Vector{Polytope}()  # global list of polytopes, empty initially

# Function to process the polytope generation and extraction for each `i`
function process_polytopes(i)
    local_polytopes_list = Vector{Polytope}()  # List to hold polytopes for this particular `i`
    try
        open("SmoothGeneration_output/SmoothGeneration_output$i") do f
            number_polytopes = 0
            polytopes_read = 0
            while !eof(f)
                # Read the number of vertices
                number_vertices = parse(Int, readline(f))
                polytopes_read += 1

                vertices_matrix = zeros(Int, number_vertices, 4)
                # Read the matrix of vertices of polytopes P
                for a in 1:number_vertices
                    vertices_string = readline(f)
                    vertex_vector = [1; parse.(Int64, split(vertices_string))]  # Append 1 to the beginning due to polymake standards
                    vertices_matrix[a, :] .= vertex_vector
                end
                # Create the polytope from its vertices
                new_polytope = polytope.Polytope(POINTS=vertices_matrix)

                if new_polytope.SMOOTH == false  # Checks that the polytope is smooth
                    for row in eachrow(new_polytope.VERTICES) 
                        println(join(row[2:end], " "))
                    end
                else
                    @assert new_polytope.NORMAL  # Checks that the polytope is normal
                end

                if new_polytope.N_LATTICE_POINTS <= MAX_LATTICE_POINTS  # Check if the polytope does not exceed the maximum number of lattice points
                    isomorphic_found = false
                    # Check if polytopes_list already has this polytope
                    for smooth_polytope in local_polytopes_list
                        if polytope.lattice_isomorphic_smooth_polytopes(smooth_polytope, new_polytope)  # Check if any of the polytopes are lattice-equivalent
                            isomorphic_found = true  # Raise flag
                            break
                        end
                    end
                    if !isomorphic_found
                        push!(local_polytopes_list, new_polytope)  # Append to the list for this particular `i`
                        number_polytopes += 1
                    end
                end
            end 
            println(number_polytopes, " smooth polytopes extracted of ", polytopes_read, " polytopes read from file SmoothGeneration_output$i")
        end
    catch e
        if isa(e, SystemError)
            println("Skipping file SmoothGeneration_output$i, does not exist/not found.")
        else
            rethrow(e)
        end    
    end
    return local_polytopes_list  # Return the list of polytopes for this `i`
end

# Run the loop in parallel
Threads.@threads for i in 4:2:32
    local_polytopes = process_polytopes(i)  # Process polytopes for the current `i`
    
    # Thread-safe way to concatenate lists. Use `lock` to ensure threads don't interfere.
    lock = ReentrantLock()  # Lock to ensure thread-safety
    lock(lock) do
        append!(polytopes_list, local_polytopes)  # Append to the global list
    end
end

# Optional: After the parallel execution, print or analyze the results
println("Total number of polytopes processed: ", length(polytopes_list))

4 smooth polytopes extracted of 4 polytopes read from file SmoothGeneration_output4
2074 smooth polytopes extracted of 10969 polytopes read from file SmoothGeneration_output6
1571 smooth polytopes extracted of 27225 polytopes read from file SmoothGeneration_output8
805 smooth polytopes extracted of 9249 polytopes read from file SmoothGeneration_output10
1355 smooth polytopes extracted of 16724 polytopes read from file SmoothGeneration_output12
498 smooth polytopes extracted of 8218 polytopes read from file SmoothGeneration_output14
312 smooth polytopes extracted of 4177 polytopes read from file SmoothGeneration_output16
37 smooth polytopes extracted of 419 polytopes read from file SmoothGeneration_output18
4 smooth polytopes extracted of 55 polytopes read from file SmoothGeneration_output20
0 smooth polytopes extracted of 9 polytopes read from file SmoothGeneration_output22
1 smooth polytopes extracted of 10 polytopes read from file SmoothGeneration_output24
Skipping file SmoothGenerat

In [4]:
#Outputting the Smooth 3-polytopes after creating the list
output_file = "prune_output" # Output Filename

if !isfile(output_file)
    touch(output_file)  # Create an empty file if it doesn't exist
end

for smooth_polytope in polytopes_list
    @assert smooth_polytope.SMOOTH
    @assert smooth_polytope.NORMAL
    
    open(output_file, "a") do file #Open the file in appending mode
        println(file, smooth_polytope.N_VERTICES)

        #Printing the contents of the matrix and avoiding the Polymake types showing up
        for row in eachrow(smooth_polytope.VERTICES) 
            println(file, join(row[2:end], " "))
        end
    end
end

In [5]:
#Performing various analyses on the produced polytopes
counter = 0
counter1 = 0
counter2 = 0
counter3 = 0
counter4 = 0

for smooth_polytope in polytopes_list
    if smooth_polytope.N_INTERIOR_LATTICE_POINTS >= 1
        counter += 1
        if smooth_polytope.N_INTERIOR_LATTICE_POINTS == 1
            counter1 += 1
        end
        if smooth_polytope.N_INTERIOR_LATTICE_POINTS == 2
            counter2 += 1
        end
        if smooth_polytope.N_INTERIOR_LATTICE_POINTS == 3
            counter3 += 1
        end
        if smooth_polytope.N_INTERIOR_LATTICE_POINTS == 4
            counter4 += 1
        end
    end
end

println(counter, " with an interior lattice point")
println(counter1, " with one interior lattice point")
println(counter2, " with two interior lattice points")
println(counter3, " with three interior lattice points")
println(counter4, " with four interior lattice points")

println(size(polytopes_list))

981 with an interior lattice point
61 with one interior lattice point
227 with two interior lattice points
404 with three interior lattice points
238 with four interior lattice points
(6661,)


In [7]:
# Initialize a dictionary to store counts of lattice points
lattice_point_counts = Dict(i => 0 for i in 4:44)

# Iterate through each polytope in the list
for P in polytopes_list
    num_lattice_points = P.N_LATTICE_POINTS
    @assert 4 <= num_lattice_points <= 44
    lattice_point_counts[num_lattice_points] += 1
end

# Now lattice_point_counts contains the number of polytopes with each number of lattice points
# from 4 to 44. To display the results, you can print the counts:

for num_points in 4:44
    println("Lattice points = ", num_points, ": ", lattice_point_counts[num_points])
end

Lattice points = 4: 1
Lattice points = 5: 0
Lattice points = 6: 1
Lattice points = 7: 1
Lattice points = 8: 3
Lattice points = 9: 4
Lattice points = 10: 6
Lattice points = 11: 5
Lattice points = 12: 12
Lattice points = 13: 10
Lattice points = 14: 17
Lattice points = 15: 14
Lattice points = 16: 29
Lattice points = 17: 21
Lattice points = 18: 39
Lattice points = 19: 30
Lattice points = 20: 54
Lattice points = 21: 42
Lattice points = 22: 63
Lattice points = 23: 56
Lattice points = 24: 94
Lattice points = 25: 75
Lattice points = 26: 113
Lattice points = 27: 91
Lattice points = 28: 154
Lattice points = 29: 103
Lattice points = 30: 186
Lattice points = 31: 140
Lattice points = 32: 247
Lattice points = 33: 180
Lattice points = 34: 292
Lattice points = 35: 227
Lattice points = 36: 353
Lattice points = 37: 256
Lattice points = 38: 411
Lattice points = 39: 335
Lattice points = 40: 541
Lattice points = 41: 431
Lattice points = 42: 651
Lattice points = 43: 556
Lattice points = 44: 817


In [10]:
p = polytopes_list[1200]
@show p.VERTICES
println(p.INTERIOR_LATTICE_POINTS)
p.VISUAL

p.VERTICES = pm::Matrix<pm::Rational>
1 0 0 0
1 0 1 0
1 3 0 0
1 3 3 0
1 2 3 0
1 0 0 1
1 0 1 1
1 3 0 1
1 3 3 1
1 2 3 1

pm::Matrix<pm::Integer>



LoadError: Exception occured at Polymake side:
unknown property Polytope<Rational>::VISUAL at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 432.
	Polymake::Core::BigObjectType::property(Polymake::Core::BigObjectType=ARRAY(0x5ff4d48), "VISUAL") called at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 714
	Polymake::Core::BigObjectType::encode_descending_path(Polymake::Core::BigObjectType=ARRAY(0x5ff4d48), "VISUAL") called at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObjectType.pm line 757
	Polymake::Core::BigObjectType::encode_read_request(Polymake::Core::BigObjectType=ARRAY(0x5ff4d48), "VISUAL") called at /home/kyle/.julia/artifacts/51aeebd7aa37184cb3796181547a7148af3bd674/share/polymake/perllib/Polymake/Core/BigObject.pm line 1551
	Polymake::Core::BigObject::